In [1]:
import sys
sys.path.append('..')

In [2]:
from synthetic_data_generation import generator
from similarity_check.SimilarityCheck import *
from synthetic_data_generation.generator import *
from utils import *
from privacy_check import *

/opt/anaconda3/envs/CRP/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


IPython not installed.


In [3]:
# define path to the data you want to test
path_test_data = "../Subsample_training.csv"

# take the comment out to see the first 10 rows of your data

# indicate which columns are categorical, and which are sensitive
cat_cols = ['Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE', 'Risk_Flag']
sensitive_cols = ["first_name", "last_name", "email", "gender", "ip_address", "nationality", "city"]

my_metadata = {
    'fields':
        {
            'Income': {'type': 'numerical', 'subtype': 'integer'},
            'Age': {'type': 'numerical', 'subtype': 'integer'},
            'Experience': {'type': 'numerical', 'subtype': 'integer'},
            'CURRENT_JOB_YRS': {'type': 'numerical', 'subtype': 'integer'},
            'CURRENT_HOUSE_YRS': {'type': 'numerical', 'subtype': 'integer'},
            'Married/Single': {'type': 'categorical'},
            'House_Ownership': {'type': 'categorical'},
            'Car_Ownership': {'type': 'categorical'},
            'Profession': {'type': 'categorical'},
            'CITY': {'type': 'categorical'},
            'STATE': {'type': 'categorical'},
            'Risk_Flag': {'type': 'boolean'}
        },
    'constraints': [],
    'model_kwargs': {},
    'name': None,
    'primary_key': None,
    'sequence_index': None,
    'entity_columns': [],
    'context_columns': []
}

data = get_data(path_test_data)
# checking that it can deal with nan values
data.iloc[3, 2] = float("nan")
print(data.head())
# create object
generator = Generator(n_epochs=100, n_samples=100, architecture='CTGAN',
                        data=data,
                        categorical_columns=cat_cols,
                        sensitive_columns=sensitive_cols)
print("Generating data")
synth_data = generator.generate().iloc[:, 2:]

   Unnamed: 0      Id     Income  Age  Experience Married/Single  \
0      189701  189702  2201763.0   70          15         single   
1      176485  176486  7159687.0   38           5         single   
2       27140   27141  4539480.0   46           3         single   
3       95936   95937        NaN   42          16         single   
4       82608   82609  6501844.0   79          14         single   

  House_Ownership Car_Ownership           Profession       CITY  \
0          rented            no              Dentist      Kulti   
1          rented            no    Computer_operator    Jalgaon   
2          rented           yes     Fashion_Designer  Eluru[25]   
3          rented            no  Biomedical_Engineer      Bhind   
4          rented           yes             Engineer     Bhopal   

            STATE  CURRENT_JOB_YRS  CURRENT_HOUSE_YRS  Risk_Flag  
0     West_Bengal               12                 14          0  
1     Maharashtra                3                 11 

In [4]:
synth_data

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,2204815.0,68,16,single,rented,no,Physician,Farrukhabad,Uttar_Pradesh,9,13,1
1,6269912.0,37,4,married,rented,no,Physician,Mehsana,Karnataka,3,10,1
2,8074205.0,34,19,single,norent_noown,yes,Chartered_Accountant,Kurnool[18],Maharashtra,2,13,1
3,3474652.0,79,15,single,norent_noown,no,Computer_hardware_engineer,Panihati,Chhattisgarh,1,10,0
4,9992133.0,77,5,married,rented,yes,Design_Engineer,Panipat,Uttar_Pradesh,5,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,4215539.0,35,10,single,norent_noown,yes,Industrial_Engineer,Ratlam,Chandigarh,2,11,0
96,1023598.0,26,11,single,rented,no,Analyst,Bardhaman,Bihar,2,14,1
97,7886694.0,79,5,single,norent_noown,no,Software_Developer,Bhubaneswar,Karnataka,6,12,0
98,9992133.0,79,20,single,rented,yes,Web_designer,Alappuzha,Andhra_Pradesh,2,14,0


In [5]:
real_data = data.drop(["Unnamed: 0", "Id"], axis = 1)

In [6]:
real_data

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,2201763.0,70,15,single,rented,no,Dentist,Kulti,West_Bengal,12,14,0
1,7159687.0,38,5,single,rented,no,Computer_operator,Jalgaon,Maharashtra,3,11,0
2,4539480.0,46,3,single,rented,yes,Fashion_Designer,Eluru[25],Andhra_Pradesh,3,11,0
3,NaN,42,16,single,rented,no,Biomedical_Engineer,Bhind,Madhya_Pradesh,5,13,0
4,6501844.0,79,14,single,rented,yes,Engineer,Bhopal,Madhya_Pradesh,13,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,4754357.0,25,0,single,rented,no,Lawyer,Pondicherry,Puducherry,0,12,0
1996,7591621.0,41,6,single,owned,no,Artist,Jaunpur,Uttar_Pradesh,3,14,1
1997,6830726.0,29,20,married,rented,yes,Secretary,Rajkot,Gujarat,3,14,1
1998,8645202.0,65,9,single,owned,no,Technician,Mango,Jharkhand,9,13,0


In [7]:
#score, pairs = generate_report(real_data, synth_data, my_metadata, dist_threshold=0.15, only_cat=False)
privacy = PrivacyCheck(real_data, synth_data, my_metadata)

In [8]:
privacy.generate_privacy_score()

Generating report...Done!


In [10]:
score, pairs = privacy.get_privacy_score()

############ SCORE ############
Privacy score:  84.81%
############ TOP 3 CLOSEST PAIRS ############
# 1. Closest pair with distance:  0.1519


|                   | Synthetic obs. (idx: 56)   | Closest real obs. (idx: 1212)   |
|:------------------|:---------------------------|:--------------------------------|
| Income            | 6197867.0                  | 6415225.0                       |
| Age               | 64                         | 54                              |
| Experience        | 16                         | 13                              |
| Married/Single    | single                     | single                          |
| House_Ownership   | rented                     | rented                          |
| Car_Ownership     | no                         | no                              |
| Profession        | Flight_attendant           | Flight_attendant                |
| CITY              | Sambalpur                  | Firozabad                       |
| STATE             | Uttar_Pradesh              | Uttar_Pradesh                   |
| CURRENT_JOB_YRS   | 5                          | 3                               |
| CURRENT_HOUSE_YRS | 12                         | 13                              |
| Risk_Flag         | 0                          | 0                               |

# 2. Closest pair with distance:  0.1532


|                   | Synthetic obs. (idx: 0)   | Closest real obs. (idx: 440)   |
|:------------------|:--------------------------|:-------------------------------|
| Income            | 2204815.0                 | 6973369.0                      |
| Age               | 68                        | 68                             |
| Experience        | 16                        | 20                             |
| Married/Single    | single                    | single                         |
| House_Ownership   | rented                    | rented                         |
| Car_Ownership     | no                        | no                             |
| Profession        | Physician                 | Physician                      |
| CITY              | Farrukhabad               | Raebareli                      |
| STATE             | Uttar_Pradesh             | Uttar_Pradesh                  |
| CURRENT_JOB_YRS   | 9                         | 11                             |
| CURRENT_HOUSE_YRS | 13                        | 13                             |
| Risk_Flag         | 1                         | 1                              |

# 3. Closest pair with distance:  0.1701


|                   | Synthetic obs. (idx: 93)   | Closest real obs. (idx: 1963)   |
|:------------------|:---------------------------|:--------------------------------|
| Income            | 8906322.0                  | 8753453.0                       |
| Age               | 74                         | 52                              |
| Experience        | 14                         | 14                              |
| Married/Single    | single                     | single                          |
| House_Ownership   | rented                     | rented                          |
| Car_Ownership     | no                         | no                              |
| Profession        | Designer                   | Geologist                       |
| CITY              | Guntur[13]                 | Guntur[13]                      |
| STATE             | Andhra_Pradesh             | Andhra_Pradesh                  |
| CURRENT_JOB_YRS   | 1                          | 7                               |
| CURRENT_HOUSE_YRS | 10                         | 11                              |
| Risk_Flag         | 0                          | 0                               |